In [1]:
import json
from pprint import pprint
import pandas as pd
import os
import re
import regex
import time
import numpy as np

In [2]:
base_path="/Users/sijiawu/Work/Thesis/Data/"

In [3]:
# Merged=pd.read_excel(base_path+'/Combined/AER_M_sco_du.xlsx')
# Merged.loc[Merged['journal']=='The American Economic Review','journal']='AER'
JOURNALS= [
    'AER', 
    'JPE', 
    'ECTA', 
    'RES', 
    'QJE'
]

#read in all processed masterlists
j_data={}
for i in JOURNALS:
    j_data[i]=pd.read_excel(base_path+'Combined/'+i+'_M_sco_du.xlsx')
#Create a batch file
j_data=pd.concat(j_data.values(), ignore_index=True)
j_data=j_data[j_data.duplicated()==False].reset_index(drop=True)


# Replace the journal names with Acronyms
j_data.loc[j_data['journal']=="Econometrica",'journal']='econometrica'
j_data.loc[j_data['journal']=='The Quarterly Journal of Economics','journal']='quarterly journal of economics'
j_data.loc[j_data['journal']=='The Review of Economic Studies','journal']='review of economic studies'
j_data.loc[j_data['journal']=='Journal of Political Economy','journal']='journal of political economy'
j_data.loc[j_data['journal']=='The American Economic Review','journal']='american economic review'

#some corrections to the issue
j_data.loc[j_data["number"]=="2023-03-04 00:00:00","number"]="3--4"
j_data.loc[j_data["number"]=="4-5","number"]="4--5"
j_data.loc[j_data["number"]=="1-2","number"]="1--2"

j_data["ID"]=j_data["URL"].str.split("/").str[-1]
j_data["title_proc"]=j_data["title"].fillna("none").astype(str).str.lower()


Merged=j_data

/var/folders/5b/5mt219qj6l552yrf3l89xgdh0000gn/T/ipykernel_72120/2610317099.py:14: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  j_data[i]=pd.read_excel(base_path+'Combined/'+i+'_M_sco_du.xlsx')


In [4]:
def open_json(filepath):
    with open(filepath) as json_data:
        d = json.load(json_data)
        json_data.close()
        return d    

In [ ]:
l_check=[]
ref_names=["full reference in Chicago referencing style",'reference','full reference','full_reference']
av=0
js=0
js_6=0
issue=[]
references={}
manu=[]
manu2=[]
ex1=[]
ex2=[]
keys=[]
jour=[]
alt=[]
alt2=[]
alt3=[]
nj=[]
probs=[]
eds=[]
count=0
book=0
al=[]
issue_papers=[]
for i in Merged.index:
    # if count>100:
    #     break
    paper_id=Merged.loc[i,"ID"]
    # if paper_id!='1881793':
    #     continue
    p_ref={"refs":[], "year_o":Merged.loc[i, "year"], "journal_o":Merged.loc[i, "journal"]} 
    filename=base_path+"new_gpt/"+Merged.loc[i,"ID"]+"_chatgpt.json"
    if os.path.exists(filename):
        count=count+1
        # print(paper_id)
        ref_count=0
        paper=open_json(filename)
        ke=paper[paper_id].keys()
        # print(ke)
        if 'choices' in ke:
            ke=['single']
        pages=[]
        for x in ke:
            if x=='single':
                content=paper[paper_id]['choices']['message']['content']
            else:
                content=paper[paper_id][x]['choices']['message']['content']
            # print(content)
            js=js+1
            ref_split=re.findall("{[^{}]*}", content)
            # print(len(ref_split))
            # print(ref_split)
            if len(ref_split)==0:
                # print(x)
                # print(content)
                pages.append(0)
            else:
                pages.append(1)

            for k in ref_split:
                ref_single=k.split("\n")
                # print(k)
                if len(ref_single)<5:
                    continue
                
                i_ref={}
               
                for r in ref_single:
                    reg_groups=re.search('\s+"([^:]*?)": "(.*)"(|,)',r)
                    if reg_groups is not None:
                        a=reg_groups.groups()
                        i_ref[a[0]]=a[1]
                        keys.append(a[0])
                ref_count+=1

                if set(["authors", "year", "title"]).issubset(list(i_ref.keys())) and (set(i_ref.keys()).intersection(ref_names)):
                    ref_label=list(set(i_ref.keys()).intersection(ref_names))[0]

                    if ((i_ref["title"]=="NA")&(i_ref["year"]=="NA"))|(i_ref["title"]=="NA")&(i_ref["authors"]=="NA")|(i_ref[ref_label]=="NA"):
                        continue
                    if i_ref[ref_label]=="NA":
                        continue
                        
                    lx=re.sub(r"[‘’]","'", i_ref["title"].lower())
                    lx=re.sub(r"[“”]",'"', lx)
                    lx=re.sub(r"''",'"',lx)
                    lx=re.sub(r'""','"',lx)
                    lx=re.sub(r'\'"\'','"',lx)
                    lx=re.sub(r"(?<=\D)(- )(?=\D)", "", lx)
                    
                    rx=re.sub(r"[“”]",'"',i_ref[ref_label].lower())
                    rx=re.sub(r"[‘’]","'", rx)
                    rx=re.sub(r"''",'"',rx)
                    rx=re.sub(r'""','"',rx)
                    rx=re.sub(r'\'"\'','"',rx)
                    rx=re.sub(r"(?<=\D)(- )(?=\D)", "", rx)
                    
                    
                    lx_x=re.sub("[()\[\]'*]",".",lx)
                    lx_x=regex.sub(r'\\(\d)','\1',lx_x)
                    rx_x=re.sub("[\[\]]",".",rx)
                    
                    l_check.append(rx)
                    
                    i_ref["status"]=True
                    if (lx not in rx):
                        r = regex.compile('(%s){e<=5}' % lx_x)
                        if r.search(rx_x) is None:
                            print("***** issue *****")
                            print(lx)
                            print(rx)
                            print(lx_x)
                            print(rx_x)
                            print("***** issue *****")
                            i_ref['status']=False

                    reg1=r'(?<!".+)([^()":0-9]+)[\(]*(\d+\D*?|na|forthcoming|undated)[\)]*[:. ]+"(.+)"[\'\"-+,\. ]+([^"\d]+?)(vol.*|\d.*)(\[.+\]|)'
                    reg2=r'(?<!".+)([^()":0-9]+)[\(]*(\d{2,5}\D{0,2}?|na|forthcoming|undated)[:. \)]+(".+|)'
                    reg3=r'(\D+).(\d+[^0-9.]*?|na).([^.]*).(.+?)(\d+):(.*)'
                    reg4=r'(?<!".+)([^()":0-9]+)[:. ]+"(.+)"[\'\"-+,\. ]+([^"\d]+)(vol.*|\d*)'
                    if regex.search(reg1,rx) is not None:
                        i_ref["alt_j"]= regex.search(reg1,rx).groups()[3]
#                         print("found")
#                         print(rx)
                        i_ref['regex']={"pattern":reg1, "type":1}
                        chap=regex.search("^chap|^chapter |^ch. | chap.|chapter", i_ref["alt_j"].strip())
                        ed=regex.search('ed\. | eds\. |edited by|editor| ed\.|\(ed\.|eds\.\)', i_ref["alt_j"].strip())
                        if "ed." in rx or "eds." in rx or "edited by" in rx or "editor" in rx:
                            book+=1
                        has_in=regex.search(r"^in .+",i_ref["alt_j"].strip())
                        http=regex.search(r'http:|https:', rx)
                        accessed_at=regex.search(r'accessed at http',rx)
                        dis_paper=regex.search("discussion paper",rx)
                        work_paper=regex.search("working paper", rx)
                        un_pub=regex.search("unpublished .*", i_ref["alt_j"])
                        report=regex.search("^report", i_ref["alt_j"].strip())
                        if chap is not None or ed is not None or has_in is not None:
                            print(chap is not None)
                            print(ed is not None)
                            print(has_in is not None)
                            
                        else:
                            print(i_ref["alt_j"])
                        if http is not None:
                            print(http is not None)
                            print(rx)
                        jour.append(i_ref["alt_j"].strip())
                    
                    elif regex.search(reg2,rx) is not None:
#                         print("found likely chicago style pattern but not journal reference pattern")
                        i_ref['regex']={"pattern":reg2, "type":2, "match_target":rx}
#                         print(regex.search(reg2,rx).groups())
                        alt.append(rx)
                    elif regex.search(reg4,rx) is not None:
                        i_ref["alt_j"]= regex.search(reg4,rx).groups()[2]
                        i_ref['regex']={"pattern":reg4, "type":4, "match_target":rx}

                        alt3.append(rx)
                    elif regex.search(reg3,rx) is not None:
                        i_ref["alt_j"]= regex.search(reg3,rx).groups()[4]
#                         print("found alternative style pattern")
                        i_ref['regex']={"pattern":reg3, "type":3, "match_target":rx}
                        alt2.append(rx)
                    

                    else:
                        nj.append(rx)
                        print("not found")
                        i_ref['regex']={"pattern":None, "type":None, "match_target":rx}
                        # print(lx_x)
                        # print(rx)
                        print("\n")
                        probs.append(paper_id)
                        print(i_ref)
#                     # t2=time.time()
# #                     print(t2-t1)
                # pprint(i_ref)
                
                p_ref["refs"].append(i_ref)
        # else:
        #     js_6+=1
        #     issue_papers.append(paper_id)
        # av=av+1
        if ((len(ke)-1)>sum(pages))&(Merged.loc[i,'year']<1985):
            print(paper_id)
            print('potential issue')
            print((len(ke)-1))
            print(sum(pages))
            print(Merged.loc[i,'year'])
                       
            print(Merged.loc[i,'journal'])

        references[paper_id]=p_ref
# #         print("\n\n")
        

In [6]:
set(keys)

{'authors', 'full_reference', 'month', 'pages', 'publisher', 'title', 'year'}

In [7]:
print(js)
print(js_6)

10254
0


In [8]:
format=[]

for i in references.keys():
    # print(i)
    temp_ref=references[i]["refs"]
    k=0
    for j in temp_ref:            
        format.append({"year_o":references[i]["year_o"], "journal_o": references[i]["journal_o"], "id_o": i, "ref_ord": k}|j)
        k=k+1

In [9]:
output=pd.DataFrame(format)
output.columns

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex'],
      dtype='object')

In [10]:
output.shape

(155782, 14)

In [11]:
output_r=output.replace("NA", np.nan)

In [12]:
output_r[output_r['status']==False].shape

(52, 14)

In [13]:
print(output_r.columns)

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex'],
      dtype='object')


In [14]:
output_f=output_r

In [15]:
output_f['f_key']='tesseract_'+str(output_f.index)

In [16]:
len(alt)
len(alt2)
len(alt3)
book
(len(nj)+172)/(len(jour) +len(alt)+len(alt3)+len(nj))

0.26010600373972587

In [17]:
print(len(probs))
p_list=list(set(probs))

data=[]
for i in references.keys():
    hal=0
    for k in references[i]["refs"]:
        if "status" not in k.keys():
            continue
        if k['status']==False:
            hal+=1
    data.append({'ID':i,'total':len(references[i]["refs"]), 'probs':probs.count(i), "hal":hal,'journal':j_data[j_data.ID==i]['journal'].values[0]})

out=pd.DataFrame.from_dict(data)

38499


In [18]:
out['rat']=out.probs/out.total
out=out.sort_values(by=["probs"],ascending=False)

In [19]:
out[out.rat>0]["journal"].value_counts()

american economic review          1496
journal of political economy       316
quarterly journal of economics     298
review of economic studies          69
econometrica                        45
Name: journal, dtype: int64

In [20]:
out["journal"].value_counts()

american economic review          2617
journal of political economy       608
quarterly journal of economics     304
econometrica                       274
review of economic studies         201
Name: journal, dtype: int64

In [21]:
j_list=list(set(jour))
j_list.sort()
len(j_list)
len(jour)
j_list=list(set(j_list))

book_chaps=0
h_link=0
man=0
mim=0
for i in range(len(j_list)):
    if j_list[i][:3]=="in ":
        # print(j_list[i])
        book_chaps+=1
        continue  
    if (j_list[i][:9]=="accessed ")|(j_list[i][:10]=="available ")|(j_list[i][:14]=="last accessed "):
        h_link+=1
        continue
    if "manuscript" in j_list[i][:15]:
        man+=1
        continue
    if "mimeo" in j_list[i]:
        mim+=1
        continue

In [22]:
publishers=list(output_f[output_f['publisher'].isna()==False]['publisher'].str.strip().str.upper().unique())
publishers.sort()
print(len(publishers))

alt_j=list(output_f[output_f['alt_j'].isna()==False]['alt_j'].str.strip().str.upper().unique())
alt_j.sort()
print(len(alt_j))


recon_pubs=alt_j+publishers
recon_pubs=list(set(recon_pubs))
recon_pubs.sort()
print(len(recon_pubs))

# new_j={}
# for i in rep_names:
#     new_j[i]=[]
#     for j in rep_names[i]:
#         new_j[i].append(j.strip().upper())
#     new_j[i]=list(set(new_j[i]))
#     new_j[i].sort()

# with open('save_v2.json', 'w') as f:
#     json.dump(new_j, f, indent=4)


18974
7184
24124


In [24]:
rep_names=open_json("./031_recon/gpt_alt_j_v2.json")
rep_names_expand={}
for i in rep_names.keys():
    for j in list(set(rep_names[i])):
        rep_names_expand[j]=i

rep_keys=rep_names_expand.keys()
for i in output_f.index:
    if (pd.isna(output_f.loc[i,"publisher"])==False):
        if (output_f.loc[i,"publisher"].strip().upper() in rep_keys):
            output_f.loc[i,"publisher_proc"]=rep_names_expand[output_f.loc[i,"publisher"].strip().upper()]
    if (pd.isna(output_f.loc[i,"alt_j"])==False):
        if (output_f.loc[i,"alt_j"].strip().upper() in rep_keys):
            output_f.loc[i,"alt_j_proc"]=rep_names_expand[output_f.loc[i,"alt_j"].strip().upper()]


In [25]:
for i in range(len(recon_pubs)):
    if recon_pubs[i] in rep_names_expand.keys():
        recon_pubs[i]=rep_names_expand[recon_pubs[i]]

In [26]:
recon_pubs=list(set(recon_pubs))
print(len(recon_pubs))

23671


In [27]:
years=list(output_f[output_f['year'].isna()==False]['year'].str.strip().unique())
years.sort()

In [28]:
years=open_json("./031_recon/gpt_years_v2.json")


In [29]:
def proc_year(x):
    y=x.replace(";",",")
    y=y.replace(" ","")
    p=[]
    for m in y.split(','):
        for n in m.split('-'):
            p.append(int(n))

    return [min(p),max(p)]

In [31]:
j_keys=['a.e.r.', 'econometrica', 'j.p.e.', 'q.j.e.', 'r.e.s.', 'ibid']
j_keys =['J. P. E.', 'A. E. R.', 'ECONOMETRICA', 'Q. J. E.', 'R. E. S.', 'THIS']

In [213]:
output_f[(output_f['publisher_proc'].isna()==False) | (output_f['alt_j_proc'].isna()==False)].shape

(26304, 17)

In [32]:
print("ensured") #publisher exists, publisher in key, alt exists, alt in key
print(output_f[(output_f["publisher_proc"].isna()==False)&(output_f["publisher_proc"].isin(j_keys)==True)&(output_f["alt_j_proc"].isna()==False)& (output_f["publisher_proc"]==output_f["alt_j_proc"])].shape)

print("no publisher or alternate picked up")
print(output_f[(output_f["publisher_proc"].isna()==True)&(output_f["alt_j_proc"].isna()==True)].shape)

print("no publisher but alternate top 5 found")
print(output_f[(output_f["publisher_proc"].isna()==True)&(output_f["alt_j_proc"].isna()==False)& (output_f["alt_j_proc"].isin(j_keys)==True)].shape)
print("no publisher but alternate journal found")
print(output_f[(output_f["publisher_proc"].isna()==True)&(output_f["alt_j_proc"].isna()==False)& (output_f["alt_j_proc"].isin(j_keys)==False)].shape)

print("publisher found that is not top 5 and no alternate")
print(output_f[(output_f["publisher_proc"].isna()==False)&(output_f["alt_j_proc"].isna()==True)& (output_f["publisher_proc"].isin(j_keys)==False)].shape)
print("publisher found that is top 5 and no alternate")
print(output_f[(output_f["publisher_proc"].isna()==False)&(output_f["alt_j_proc"].isna()==True)& (output_f["publisher_proc"].isin(j_keys)==True)].shape)

print("publisher found that is not top 5 and alternate found that is top 5")
print(output_f[(output_f["publisher_proc"].isna()==False)&(output_f["alt_j_proc"].isna()==False)& (output_f["alt_j_proc"].isin(j_keys)==True)& (output_f["publisher_proc"].isin(j_keys)==False)].shape)
print("publisher found that is top 5 and alternate found that is top 5")
print(output_f[(output_f["publisher_proc"].isna()==False)&(output_f["alt_j_proc"].isna()==False)& (output_f["alt_j_proc"].isin(j_keys)==True)& (output_f["publisher_proc"].isin(j_keys)==True)].shape)

print("publisher found that is not top 5 and alt is found but alt is not in top 5")
print(output_f[(output_f["publisher_proc"].isna()==False)&(output_f["alt_j_proc"].isna()==False)& (output_f["alt_j_proc"].isin(j_keys)==False)& (output_f["publisher_proc"].isin(j_keys)==False)].shape)
print("publisher found that is top 5 and alt is found that is not in top 5")
print(output_f[(output_f["publisher_proc"].isna()==False)&(output_f["alt_j_proc"].isna()==False)& (output_f["alt_j_proc"].isin(j_keys)==False)& (output_f["publisher_proc"].isin(j_keys)==True)].shape)


ensured
(4198, 17)
no publisher or alternate picked up
(129478, 17)
no publisher but alternate top 5 found
(4022, 17)
no publisher but alternate journal found
(0, 17)
publisher found that is not top 5 and no alternate
(0, 17)
publisher found that is top 5 and no alternate
(18084, 17)
publisher found that is not top 5 and alternate found that is top 5
(0, 17)
publisher found that is top 5 and alternate found that is top 5
(4198, 17)
publisher found that is not top 5 and alt is found but alt is not in top 5
(0, 17)
publisher found that is top 5 and alt is found that is not in top 5
(0, 17)


In [33]:
4198+4022+18084

26304

In [43]:
match_a=output_f[(output_f["publisher_proc"].isna()==False)&(output_f["publisher_proc"].isin(j_keys)==True)& (output_f["publisher_proc"]==output_f["alt_j_proc"])].reset_index(drop=True)
match_b=output_f[(output_f["publisher_proc"].isna()==False)&(output_f["alt_j_proc"].isna()==True)& (output_f["publisher_proc"].isin(j_keys)==True)].reset_index(drop=True)
match_c=output_f[(output_f["publisher_proc"].isna()==True)&(output_f["alt_j_proc"].isna()==False)& (output_f["alt_j_proc"].isin(j_keys)==True)].reset_index(drop=True)


In [44]:
match_a.shape

(4198, 17)

In [45]:
output_f['status'].value_counts()

True     154102
False        52
Name: status, dtype: int64

In [46]:
match_b.shape

(18084, 17)

In [47]:
match_c.shape

(4022, 17)

## Reconciliation

In [48]:
from difflib import SequenceMatcher

# Utility function to compute similarity
def similar(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

transform={
    "Q. J. E.":'quarterly journal of economics', 
    'R. E. S.':'review of economic studies',
    'ECONOMETRICA':'econometrica', 
    'J. P. E.':'journal of political economy',
    "A. E. R.":'american economic review'
}

def construct(j_data, journal,year,use_year, title):
    temp=j_data[(j_data["journal"]==journal)&(j_data["year"]<=(year+2))&(j_data["year"]>=(use_year-10))]["title_proc"].apply(lambda y: similar(y, title))
    o=temp[temp>=(max(temp)-0.15)]
    return {"index": list(o.index), "m_val":list(o)}

In [49]:
def output_proc(match_in, years,outfile_name, use_j):
    match_in["use_year"]='0'
    match_in["use_journal"]=''
    match_in["use_title"]=''
    match_in=match_in.fillna('0')
    for i in match_in.index:
        # print(match_in.loc[i,'year'].strip())
        match_in.loc[i,'year_proc']=years[match_in.loc[i,'year'].strip()]
        year_temp=match_in.loc[i, 'year_proc']
        if ("," in year_temp)|("-" in year_temp):
            match_in.loc[i, "year_latest"]=proc_year(year_temp)[1]
            match_in.loc[i, "year_earliest"]=proc_year(year_temp)[0]
        elif year_temp in ["FORTHCOMING"]:
            match_in.loc[i, "year_latest"]=int(match_in.loc[i, 'year_o'])+2
            match_in.loc[i, "year_earliest"]=int(match_in.loc[i, 'year_o'])
        elif year_temp in ["NA", "NONE", "VARIOUS"]:
            match_in.loc[i, "year_earliest"]='0'
            match_in.loc[i, "year_latest"]='0'
        else:
            match_in.loc[i, "year_latest"]=match_in.loc[i, 'year_proc']
            match_in.loc[i, "year_earliest"]=match_in.loc[i, 'year_proc']
    print(list(match_in['year_latest'].unique()))
    print(match_in.columns)
    # match_a=pd.read_excel("match_a.xlsx")
    for i in match_in.index:

        use_year=int(match_in.loc[i,'year_o'])+2
        use_journal=''
        if match_in.loc[i,use_j] in ['CURRENT', "THIS"]:
            use_journal=match_in.loc[i, 'journal_o']
        elif match_in.loc[i,use_j]=='ibid':
            if match_in[(match_in['ref_ord']==(match_in.loc[i,'ref_ord']-1)) & (match_in['id_o']==match_in.loc[i,'id_o'])].shape[0]==1:
                # print(i)
                use_journal=list(match_in.loc[(match_in['ref_ord']==(match_in.loc[i,'ref_ord']-1)) & (match_in['id_o']==match_in.loc[i,'id_o']), 'use_journal'])[0]
                # print(use_journal)
            else:
                use_journal=None
                continue
        else:
            use_journal=transform[match_in.loc[i,use_j]]
        
        use_title=match_in.loc[i,'title'].lower()
        match_in.loc[i,"use_title"]=use_title
        match_in.loc[i,"use_journal"]=use_journal
        match_in.loc[i,"use_year"]=use_year

        # print(construct(j_data, use_journal,use_year,use_title))
    print(match_in[use_j].value_counts())
    # match_a[0:100].apply(lambda x:construct(j_data, x['use_journal'],x['use_year'],x['use_title']), axis=1)
    match_in.to_excel(outfile_name, index=False)

In [52]:
output_proc(match_a, open_json("./031_recon/gpt_years_v2.json"),"./031_recon/gpt_match_a_v2.xlsx", 'publisher_proc')
output_proc(match_b, open_json("./031_recon/gpt_years_v2.json"),"./031_recon/gpt_match_b_v2.xlsx", 'publisher_proc')
output_proc(match_c, open_json("./031_recon/gpt_years_v2.json"),"./031_recon/gpt_match_c_v2.xlsx", 'alt_j_proc')


['2016', '2006', '2017', '2020', '2018', '1998', '2005', '2009', '2000', '2013', '1999', '1989', '2012', '2014', '2001', '2011', '1950', '1994', '2007', '2019', '1995', '2004', '1991', '1987', '1980', '2010', '1992', '1996', '2015', '2002', '1969', '1943', '2008', '1982', '1997', '1993', '1979', '1960', '1958', '1986', '2003', '1990', '1977', '1988', '1985', '1941', '1983', '1965', '1972', '1984', '1976', '1978', '1971', '1933', '1970', '1949', '1967', '1968', '1981', '1975', '1973', '1961', '1963', '1962', '1955', '1956', '1925', '1953', '1948', '1954', '1959', '1966', '1974', '0', '1964', '1951', '1928', '1945', '1938', '1919', '1911', '1939', '1924', '1913', '1937', '1931', '1952', '1936', '1957', 1954, '1934', '1892']
Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex', 'f_key', 'publisher_proc', 'alt_j_proc', 'use_year',
       'use_journal', 'use_title', 'year_proc',

In [50]:
# years=list(match_d['year'].fillna('0').unique())
# year_struc={}
# for i in years:
#     year_struc[i]=i
# with open('years_d.json', 'w') as f:
#     json.dump(year_struc, f, indent=4)

In [53]:
matches_a=pd.DataFrame(open_json("./031_recon/outcomes_match_a_v2_full.json"))
matches_b=pd.DataFrame(open_json("./031_recon/outcomes_match_b_v2_full.json"))
matches_c=pd.DataFrame(open_json("./031_recon/outcomes_match_c_v2_full.json"))


In [54]:
counter_a=[]
for i in matches_a.index:
    inds=matches_a.loc[i,'outcome']['index']
    m_vals=matches_a.loc[i,'outcome']['m_val']
    # print(inds)
    # print(m_vals)
    counter_a.append(len(inds))
    # if len(inds)==1:
        # print(j_data.iloc[inds])

counter_b=[]
for i in matches_b.index:
    inds=matches_b.loc[i,'outcome']['index']
    m_vals=matches_b.loc[i,'outcome']['m_val']
    # print(inds)
    # print(m_vals)
    counter_b.append(len(inds))

counter_c=[]
for i in matches_c.index:
    inds=matches_c.loc[i,'outcome']['index']
    m_vals=matches_c.loc[i,'outcome']['m_val']
    # print(inds)
    # print(m_vals)
    counter_c.append(len(inds))


In [55]:
matches_a['btch']='a'
matches_b['btch']='b'
matches_c['btch']='c'


In [56]:
fullset_article=pd.concat([matches_a,
                            matches_b,
                            matches_c
                            ]).reset_index(drop=True)

In [57]:
fullset_article.columns

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex', 'f_key', 'publisher_proc', 'alt_j_proc', 'use_year',
       'use_journal', 'use_title', 'year_proc', 'year_latest', 'year_earliest',
       'outcome', 'btch'],
      dtype='object')

In [58]:
from fuzzywuzzy import fuzz

In [59]:
def alias(names):
    a_fin=[]
    b_fin=[]
    # print(names)
    if pd.isna(names):
        return None, None, None
    a_temp=names.lower().split(' and ')
    for k in range(len(a_temp)):
        fn=a_temp[k].split(' ')
        init=''
        for l in range(len(fn)-1):
            init+=fn[l][0]+". "
        a_fin.append(init+fn[-1])
        b_fin.append(fn[0][0]+'. '+fn[-1])
    return "; ".join(a_fin), ", ".join(b_fin), " ".join(a_fin), " ".join(b_fin)

In [60]:
fullset_article.columns

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex', 'f_key', 'publisher_proc', 'alt_j_proc', 'use_year',
       'use_journal', 'use_title', 'year_proc', 'year_latest', 'year_earliest',
       'outcome', 'btch'],
      dtype='object')

In [61]:
j_data["at_var"]=j_data.apply(lambda x: alias(x['author']), axis=1)

In [ ]:
fullset_article['match_url']=None
found=[]
confirmed={
    1:[0,0],
    2:[0,0],
    3:[0,0],
    4:[0,0],
    5:[0,0],
}
vary=0
vary2=0
bad=0
re=0
for i in fullset_article.index:
    # if i ==200:
    #     break
    count=len(fullset_article.loc[i,"outcome"]['index'])
    found.append(count)
    test=pd.concat([j_data.loc[fullset_article.loc[i,"outcome"]['index'], ["title",'author','URL','year','at_var']].reset_index(drop=True),pd.DataFrame(fullset_article.loc[i,"outcome"])], axis=1)
    test=test[test['author'].isna()==False].reset_index(drop=True)
    if (test.shape[0]==0) | (count==0):
        fullset_article.loc[i,'match_url']="CHECKREQ"
        print("no_match")
        bad+=1
        continue
    elif count==1:
        a=test.loc[0,'author'].lower()
        b=fullset_article.loc[i,"authors"].lower().replace("; "," ")
        c=fuzz.token_sort_ratio(a,b)
        d=test.loc[0,'at_var']
        m=test.loc[0,'m_val']

        if (m>=0.7)&((c>=80)|(fuzz.token_sort_ratio(b,d[2])>=80)|(fuzz.token_sort_ratio(b,d[3])>=80)):
            fullset_article.loc[i,'match_url']=list(test['URL'])[0]

        elif (m>=0.7)&(int(fullset_article.loc[i,'year_latest'])==int(test.loc[0,'year'])):
            fullset_article.loc[i,'match_url']=list(test['URL'])[0]
        elif ((fullset_article.loc[i,'year_latest']=='0')|(fullset_article.loc[i,'year_latest']==0)) & ((fullset_article.loc[i,'authors']==0)|(fullset_article.loc[i,'authors']=='0'))&(m>=0.95):
            fullset_article.loc[i,'match_url']=list(test['URL'])[0]
        else:
            print("single")
            print(fullset_article.loc[i,"outcome"]['m_val'])
            print(fullset_article.loc[i,["use_title",'year_proc', 'authors','btch']])
            print(j_data.loc[fullset_article.loc[i,"outcome"]['index'],['title_proc','year','author']])
            fullset_article.loc[i,'match_url']="CHECKREQ"

    elif count>1:
        if sum(test["m_val"]==1)==1:
            a=list(test[test["m_val"]==1]['author'])[0]
            b=fullset_article.loc[i,"authors"]
            d=list(test[test["m_val"]==1]['at_var'])[0]
            if pd.isna(a)==True:
                fullset_article.loc[i,'match_url']="CHECKREQ"
                continue
            if (fuzz.token_sort_ratio(a.lower(),b.lower())>=80)|(fuzz.token_sort_ratio(b,d[2])>=80)|(fuzz.token_sort_ratio(b,d[3])>=80):
                # confirmed[count][0]+=1
                fullset_article.loc[i,'match_url']=list(test['URL'])[0]
                continue

        if (sum(test["m_val"]>0.65)>0):
            if sum(test["year"]==fullset_article.loc[i,"year_proc"])==1:
                print("year match")
                a=similar(str(list(test[test["year"]==fullset_article.loc[i,"year_proc"]]['authors'])[0]), str(fullset_article.loc[i,"authors"]))
                b=similar(str(list(test[test["year"]==fullset_article.loc[i,"year_proc"]]['authors'])[0]).split(' ')[-1], str(fullset_article.loc[i,"authors"]).split(' ')[-1])
                if (a>0.8) or (b>0.8):
                    fullset_article.loc[i,'match_url']=list(test[test["year"]==fullset_article.loc[i,"year_proc"]]["URL"])[0]
                    vary2+=1
                else:
                    fullset_article.loc[i,'match_url']="CHECKREQ"
            vary+=1
            
        else:
            print("unlikely match")
            fullset_article.loc[i,'match_url']="CHECKREQ"
            bad+=1

In [63]:
fullset_article[fullset_article["match_url"]=="CHECKREQ"]['btch'].value_counts()

b    260
a     38
c     32
Name: btch, dtype: int64

In [64]:
fullset_article[fullset_article["match_url"]=="CHECKREQ"].shape[0]/fullset_article.shape[0]

0.011072339283317675

In [65]:
len(fullset_article["match_url"].unique())

9076

In [66]:
fullset_article['btch'].value_counts()

b    18084
a     7698
c     4022
Name: btch, dtype: int64

In [67]:
fullset_article[fullset_article["match_url"]!="CHECKREQ"].shape

(29474, 26)

In [68]:
def proc_url(x):
    if pd.isna(x):
        return None
    elif x=="CHECKREQ":
        return None
    else:
        return x.split('/')[-1]

In [69]:
fullset_article['match_id']=fullset_article.apply(lambda x: proc_url(x['match_url']), axis=1)

In [70]:
def construct(j_data, journal,year,title):
    if pd.isna(journal)==True:
        return {'index':[],"m_val":[]}
    if journal=="":
        return {'index':[],"m_val":[]}
    temp=j_data[(j_data["journal"]==journal)&(j_data["year"]<=(year+2))&(j_data["year"]>=(year-20))]["title_proc"].apply(lambda y: similar(y, title))
    if len(temp)>0:
        o=temp[temp>=(max(temp)-0.15)]
        return {"index": list(o.index), "m_val":list(o)}
    else:
        return {'index':[],"m_val":[]}

In [71]:
fullset_article.columns

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex', 'f_key', 'publisher_proc', 'alt_j_proc', 'use_year',
       'use_journal', 'use_title', 'year_proc', 'year_latest', 'year_earliest',
       'outcome', 'btch', 'match_url', 'match_id'],
      dtype='object')

In [80]:
# j_data[['issue_url', 'URL', 'journal', 'number', 'publisher', 'title',
#         'volume', 'year', 'abstract', 'author', 'pages',
#        'reviewed-author', 'uploaded', 'author_split', 'title_10',
#        'content_type', 'ID',
#        'title_proc']].to_excel('jstor_data_v2.xlsx', index=False)

In [72]:
new_df = pd.merge(
    left=output_f, 
    right=fullset_article[['year_o', 'journal_o', 'id_o', 'ref_ord','use_year', 'use_journal', 'use_title', 'year_proc',
       'year_latest','year_earliest', 'outcome', 'btch', 'match_url', 'match_id']],
    how='left',
    left_on=['year_o', 'journal_o', 'id_o', 'ref_ord'],
    right_on=['year_o', 'journal_o', 'id_o', 'ref_ord'],
)

In [73]:
new_df.columns

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex', 'f_key', 'publisher_proc', 'alt_j_proc', 'use_year',
       'use_journal', 'use_title', 'year_proc', 'year_latest', 'year_earliest',
       'outcome', 'btch', 'match_url', 'match_id'],
      dtype='object')

In [74]:
output_f.columns

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex', 'f_key', 'publisher_proc', 'alt_j_proc'],
      dtype='object')

In [75]:
new_df['f_key']='tesseract_'+new_df.index.astype(str)

In [76]:
new_df.columns

Index(['year_o', 'journal_o', 'id_o', 'ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
       'regex', 'f_key', 'publisher_proc', 'alt_j_proc', 'use_year',
       'use_journal', 'use_title', 'year_proc', 'year_latest', 'year_earliest',
       'outcome', 'btch', 'match_url', 'match_id'],
      dtype='object')

In [77]:
new_df.to_excel('./031_recon/refs_post_1970_v2.xlsx', index=False)

In [78]:
full_article_sub=new_df[(new_df['id_o']!=new_df['match_id'])&(new_df['match_id'].isna()==False)].reset_index(drop=True).drop_duplicates(subset = ['id_o', 'match_id'], keep='first').reset_index(drop=True)[['id_o', 'match_id','f_key']]

In [80]:
full_article_sub.to_excel('./031_recon/network_cit_post_v2.xlsx', index=False)